In [2]:
import os
import pandas as pd
import numpy as np

import polars as pl

In [3]:
data_dir = "~/siads696/data"

In [4]:
# df_img_embed = pd.read_csv(os.path.join(data_dir, "tl_features.csv"))
# df_img_embed.info()

In [20]:
df_pl_img_embed = pl.read_csv(os.path.join(data_dir, "tl_features.csv"))
print(df_pl_img_embed)

shape: (13_200, 3_068)
┌────────────┬───────────┬────────────┬──────┬───┬────────────┬────────────┬───────────┬───────────┐
│ TableNumbe ┆ Replicate ┆ smiles     ┆ moa  ┆ … ┆ Z1020_Acti ┆ Z1021_Acti ┆ Z1022_Act ┆ Z1023_Act │
│ r          ┆ ---       ┆ ---        ┆ ---  ┆   ┆ n          ┆ n          ┆ in        ┆ in        │
│ ---        ┆ i64       ┆ str        ┆ str  ┆   ┆ ---        ┆ ---        ┆ ---       ┆ ---       │
│ i64        ┆           ┆            ┆      ┆   ┆ f64        ┆ f64        ┆ f64       ┆ f64       │
╞════════════╪═══════════╪════════════╪══════╪═══╪════════════╪════════════╪═══════════╪═══════════╡
│ 4          ┆ 1         ┆ FC1=CNC(=O ┆ null ┆ … ┆ -0.321486  ┆ 1.2154524  ┆ -0.089709 ┆ -0.413298 │
│            ┆           ┆ )NC1=O     ┆      ┆   ┆            ┆            ┆           ┆           │
│ 4          ┆ 1         ┆ FC1=CNC(=O ┆ null ┆ … ┆ -0.629341  ┆ 2.0029707  ┆ 0.4161029 ┆ -0.491764 │
│            ┆           ┆ )NC1=O     ┆      ┆   ┆            ┆     

In [21]:
plates = df_pl_img_embed.select(
    pl.col("Image_PathName_DAPI").str.extract(r".*\/(.+)$", group_index=1).alias("plate"),
    pl.col("Image_PathName_DAPI").str.extract(r"(Week.+)_.+", group_index=1).alias("week"),
    pl.col("Image_PathName_DAPI").str.extract(r".+_(\d+)", group_index=1).alias("plate_no"),
)
print(plates)

shape: (13_200, 3)
┌─────────────┬───────┬──────────┐
│ plate       ┆ week  ┆ plate_no │
│ ---         ┆ ---   ┆ ---      │
│ str         ┆ str   ┆ str      │
╞═════════════╪═══════╪══════════╡
│ Week4_27481 ┆ Week4 ┆ 27481    │
│ Week4_27481 ┆ Week4 ┆ 27481    │
│ Week4_27481 ┆ Week4 ┆ 27481    │
│ Week4_27481 ┆ Week4 ┆ 27481    │
│ Week4_27521 ┆ Week4 ┆ 27521    │
│ …           ┆ …     ┆ …        │
│ Week9_39301 ┆ Week9 ┆ 39301    │
│ Week9_39301 ┆ Week9 ┆ 39301    │
│ Week9_39301 ┆ Week9 ┆ 39301    │
│ Week9_39301 ┆ Week9 ┆ 39301    │
│ Week9_39301 ┆ Week9 ┆ 39301    │
└─────────────┴───────┴──────────┘


In [22]:
df_pl_img_embed = pl.concat(
    [
        plates,
        df_pl_img_embed,
    ],
    how="horizontal",
)
print(df_pl_img_embed.select(pl.col("plate_no", "Image_PathName_DAPI")))

shape: (13_200, 2)
┌──────────┬─────────────────────────────────┐
│ plate_no ┆ Image_PathName_DAPI             │
│ ---      ┆ ---                             │
│ str      ┆ str                             │
╞══════════╪═════════════════════════════════╡
│ 27481    ┆ /scratch/siads696s24_class_roo… │
│ 27481    ┆ /scratch/siads696s24_class_roo… │
│ 27481    ┆ /scratch/siads696s24_class_roo… │
│ 27481    ┆ /scratch/siads696s24_class_roo… │
│ 27521    ┆ /scratch/siads696s24_class_roo… │
│ …        ┆ …                               │
│ 39301    ┆ /scratch/siads696s24_class_roo… │
│ 39301    ┆ /scratch/siads696s24_class_roo… │
│ 39301    ┆ /scratch/siads696s24_class_roo… │
│ 39301    ┆ /scratch/siads696s24_class_roo… │
│ 39301    ┆ /scratch/siads696s24_class_roo… │
└──────────┴─────────────────────────────────┘


In [23]:
print(plates.sort("plate_no").unique(maintain_order=True))

shape: (55, 3)
┌──────────────┬────────┬──────────┐
│ plate        ┆ week   ┆ plate_no │
│ ---          ┆ ---    ┆ ---      │
│ str          ┆ str    ┆ str      │
╞══════════════╪════════╪══════════╡
│ Week1_22123  ┆ Week1  ┆ 22123    │
│ Week1_22141  ┆ Week1  ┆ 22141    │
│ Week1_22161  ┆ Week1  ┆ 22161    │
│ Week1_22361  ┆ Week1  ┆ 22361    │
│ Week1_22381  ┆ Week1  ┆ 22381    │
│ …            ┆ …      ┆ …        │
│ Week9_39283  ┆ Week9  ┆ 39283    │
│ Week9_39301  ┆ Week9  ┆ 39301    │
│ Week10_40111 ┆ Week10 ┆ 40111    │
│ Week10_40115 ┆ Week10 ┆ 40115    │
│ Week10_40119 ┆ Week10 ┆ 40119    │
└──────────────┴────────┴──────────┘


In [24]:
out = df_pl_img_embed.select(
    pl.col("*").exclude("plate",
                        "week",
                        "TableNumber", 
                        "Image_FileName_DAPI",
                        "Image_PathName_DAPI",
                        "Image_FileName_Tubulin",
                        "Image_PathName_Tubulin",
                        "Image_FileName_Actin",
                        "Image_PathName_Actin",
                        "Image_Metadata_Plate_DAPI",
                        "Image_Metadata_Well_DAPI",
                        "Replicate",
                        "Image_Metadata_Concentration")
                        
)
print(out)

shape: (13_200, 3_065)
┌──────────┬────────────┬──────┬───────────┬───┬────────────┬────────────┬────────────┬────────────┐
│ plate_no ┆ smiles     ┆ moa  ┆ Z000_DAPI ┆ … ┆ Z1020_Acti ┆ Z1021_Acti ┆ Z1022_Acti ┆ Z1023_Acti │
│ ---      ┆ ---        ┆ ---  ┆ ---       ┆   ┆ n          ┆ n          ┆ n          ┆ n          │
│ str      ┆ str        ┆ str  ┆ f64       ┆   ┆ ---        ┆ ---        ┆ ---        ┆ ---        │
│          ┆            ┆      ┆           ┆   ┆ f64        ┆ f64        ┆ f64        ┆ f64        │
╞══════════╪════════════╪══════╪═══════════╪═══╪════════════╪════════════╪════════════╪════════════╡
│ 27481    ┆ FC1=CNC(=O ┆ null ┆ 0.000045  ┆ … ┆ -0.321486  ┆ 1.2154524  ┆ -0.089709  ┆ -0.413298  │
│          ┆ )NC1=O     ┆      ┆           ┆   ┆            ┆            ┆            ┆            │
│ 27481    ┆ FC1=CNC(=O ┆ null ┆ 0.000222  ┆ … ┆ -0.629341  ┆ 2.0029707  ┆ 0.4161029  ┆ -0.491764  │
│          ┆ )NC1=O     ┆      ┆           ┆   ┆            ┆       

In [26]:
out.write_csv(os.path.join(data_dir, "bbbc021_image_embed_compact.csv"))